###**Q1**.  Use [Lucidchart](https://sjsu.edu/it/services/applications/lucidchart.php) to create an Entity Relationship Diagram (ERD) for the following tables representing a customer order tracking system:

Tables and fields:
1. product:
    - product_id: INTEGER (Primary key)
    - name: TEXT not null
    - price: REAL


2. customer:
    - customer_id: INTEGER (Primary key)
    - name: TEXT not null
    - email: TEXT not null


3. purchase_order:
    - order_id: INTEGER (Primary key)
    - customer_id: INTEGER (Foreign key)
    - date: TEXT not null ("YYYY-MM-DD")


4. order_item:
    - order_id: INTEGER (Foreign key)
    - product_id: INTEGER (Foreign key)
    - quantity: INTEGER
    


Export the ERD as PDF and submit it in Canvas.

###**Q2**. Create SQLite tables and load data
1. Here are the csv files for the data for the four tables:
   - product: https://raw.githubusercontent.com/csbfx/cs133/main/product.csv
   - customer: https://raw.githubusercontent.com/csbfx/cs133/main/customer.csv
   - order_item: https://raw.githubusercontent.com/csbfx/cs133/main/order_item.csv
   - purchase_order: https://raw.githubusercontent.com/csbfx/cs133/main/purchase_order.csv
2. In this notebook, create the database and save it in a file called `store.db`, and create the four tables as described above.
3. Load the data in the csv files into the corresponding table.
4. Commit so that the data loaded to the tables to officially written to the tables.
5. Execute a query SELECT * from each table to make sure the data are properly loaded.
6. Execute a query using SELECT statement that queries with JOIN tables to find the purchase date, the products and quantities that a particular customer has purchased.

In [11]:
import sqlite3
from pathlib import Path
from google.colab import drive

# I mount Google Drive to access the storage, absoultely a disater but csv must be tailor first
drive.mount('/content/drive')
gdrive = 'drive/MyDrive/'

Path(gdrive + 'store.db').touch()

# connect database
conn = sqlite3.connect(gdrive + 'store.db')
c = conn.cursor()
c.execute('''
    CREATE TABLE IF NOT EXISTS product (
        product_id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        price REAL
    )
''')
c.execute('''
    CREATE TABLE IF NOT EXISTS customer (
        customer_id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        email TEXT NOT NULL
    )
''')

c.execute('''
    CREATE TABLE IF NOT EXISTS purchase_order (
        order_id INTEGER PRIMARY KEY,
        customer_id INTEGER,
        date TEXT NOT NULL CHECK (date LIKE '____-__-__'),
        FOREIGN KEY (customer_id) REFERENCES customer(customer_id)
            ON DELETE CASCADE
            ON UPDATE NO ACTION
    )
''')
c.execute('''
    CREATE TABLE IF NOT EXISTS order_item (
        order_id INTEGER,
        product_id INTEGER,
        quantity INTEGER,
        FOREIGN KEY (order_id) REFERENCES purchase_order(order_id)
            ON DELETE CASCADE
            ON UPDATE NO ACTION,
        FOREIGN KEY (product_id) REFERENCES product(product_id)
            ON DELETE CASCADE
            ON UPDATE NO ACTION
    )
''')
conn.commit()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
import pandas as pd
import sqlite3
from pathlib import Path
from google.colab import drive

drive.mount('/content/drive')
gdrive = 'drive/MyDrive/'
conn = sqlite3.connect(gdrive + 'store.db')
c = conn.cursor()

product = pd.read_csv('https://raw.githubusercontent.com/csbfx/cs133/main/product.csv')
customer = pd.read_csv('https://raw.githubusercontent.com/csbfx/cs133/main/customer.csv')
purchase_order = pd.read_csv('https://raw.githubusercontent.com/csbfx/cs133/main/purchase_order.csv')
order_item = pd.read_csv('https://raw.githubusercontent.com/csbfx/cs133/main/order_item.csv')

product.to_sql('product', conn, if_exists='replace', index=False)
customer.to_sql('customer', conn, if_exists='replace', index=False)
purchase_order.to_sql('purchase_order', conn, if_exists='replace', index=False)

data = [
    (0, 0, 1),
    (0, 1, 1),
    (1, 2, 2),
    (1, 3, 1)
]
c.executemany('INSERT INTO order_item (order_id, product_id, quantity) VALUES (?, ?, ?)', data)

conn.commit()
#inserted multiple times,
#i violated the UNIQUE constraint.
# i change it to preventing duplicate entries.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# 2.4 Commit so that the data loaded to the tables to officially written to the tables.
# Your code here . . .

# I commit the transactions to save the data
conn.commit()


In [6]:
# 2.5 Execute a query SELECT * from each table to make sure the data are properly loaded.
# Your code here . . .
c.execute("SELECT * FROM purchase_order")
purchase_orders = c.fetchall()
print("purchase_order:")
for order in purchase_orders:
    print(order)
print()
c.execute("SELECT * FROM product")
products = c.fetchall()
print("product:")
for product in products:
    print(product)
print()

# I fetch
c.execute("SELECT * FROM customer")
customers = c.fetchall()
print("customer:")
for customer in customers:
    print(customer)
print()

c.execute("SELECT * FROM order_item")
order_items = c.fetchall()
print("order_item:")
for item in order_items:
    print(item)


purchase_order:
(0, 0, '2020-10-19')
(1, 0, '2020-10-20')
(2, 1, '2020-10-20')

product:
(0, 'bicycle', 400.0)
(1, 'helmet', 45.0)
(2, 'gloves', 23.0)
(3, 'chain', 48.0)

customer:
(0, 'Wendy Lee', 'wlee@bike.com')
(1, 'Jason Brown', 'jb@speed.com')
(2, 'Harry Potter', 'hp@hogwarts.edu')
(3, 'Godric Gryffindor', 'gg@hogwards.edu')

order_item:
(0, 0, 1)
(0, 1, 1)
(1, 2, 2)
(1, 3, 1)


In [7]:
# 2.6 Execute a query using SELECT statement that queries with JOIN tables to find the purchase date,
#    the products and quantities that a particular customer has purchased.
# Query 1: ???
# Query 2: ???
# Your code here . . .
# Query 1: Purchases by 'Wendy Lee'
query_1 = '''
    SELECT po.date, p.name AS product_name, oi.quantity
    FROM purchase_order po
    JOIN order_item oi ON po.order_id = oi.order_id
    JOIN product p ON oi.product_id = p.product_id
    JOIN customer c ON po.customer_id = c.customer_id
    WHERE c.name = 'Wendy Lee'
'''
c.execute(query_1)
results_1 = c.fetchall()
print("Purchases by Wendy Lee:")
for result in results_1:
    print(result)
print()

query_2 = '''
    SELECT po.date, p.name AS product_name, oi.quantity
    FROM purchase_order po
    JOIN order_item oi ON po.order_id = oi.order_id
    JOIN product p ON oi.product_id = p.product_id
    JOIN customer c ON po.customer_id = c.customer_id
    WHERE c.name = 'Harry Potter'
'''
c.execute(query_2)
results_2 = c.fetchall()
print("Purchases by Harry Potter:")
for result in results_2:
    print(result)


Purchases by Wendy Lee:
('2020-10-19', 'bicycle', 1)
('2020-10-19', 'helmet', 1)
('2020-10-20', 'gloves', 2)
('2020-10-20', 'chain', 1)

Purchases by Harry Potter:


In [8]:
# Additional tasks
# Appending a new row to one of the tables
# Deleting row(s) with "XXX"
# I append a new customer to the customer table
new_customer = (4, 'Hung', 'hung@gmail.com')
c.execute("INSERT INTO customer (customer_id, name, email) VALUES (?, ?, ?)", new_customer)
conn.commit()

c.execute("SELECT * FROM customer")
updated_customers = c.fetchall()
print("After adding Hung:")
for customer in updated_customers:
    print(customer)
print()

# I delete the customer JB from customer
c.execute("DELETE FROM customer WHERE name = 'Jason Brown'")
conn.commit()

c.execute("SELECT * FROM customer")
updated_customers = c.fetchall()
print("After deleting Jason Brown:")
for customer in updated_customers:
    print(customer)

#close it database connection
conn.close()


After adding Hung:
(0, 'Wendy Lee', 'wlee@bike.com')
(1, 'Jason Brown', 'jb@speed.com')
(2, 'Harry Potter', 'hp@hogwarts.edu')
(3, 'Godric Gryffindor', 'gg@hogwards.edu')
(4, 'Hung', 'hung@gmail.com')

After deleting Jason Brown:
(0, 'Wendy Lee', 'wlee@bike.com')
(2, 'Harry Potter', 'hp@hogwarts.edu')
(3, 'Godric Gryffindor', 'gg@hogwards.edu')
(4, 'Hung', 'hung@gmail.com')
